In [2]:
import sys
sys.path.append("../src/") 

import model_manipulation as mm

import cobra
import libsbml
import riptide
from cobra import Model, Reaction, Metabolite
import csv
import pandas as pd
import copy


In [3]:
ios2164_orig = cobra.io.read_sbml_model("../model/iOS2164_rewritten.xml")
rnaseq = pd.read_csv('../data/KitaakeLCM_rawcounts.csv')
#According to the  tool's author, the best way to merge is simply to generate 

In [4]:
mcell_exp = riptide.read_transcription_file("../data/m_abundance.csv", sep = ",", header = True)
bscell_exp = riptide.read_transcription_file("../data/bs_abundance.csv", sep = ",", header = True)
vcell_exp = riptide.read_transcription_file("../data/v_abundance.csv", sep = ",", header = True)

In [5]:
ios2164_bs = copy.deepcopy(ios2164_orig)

for met in ios2164_bs.metabolites:
    if met.compartment != "e": #Extracellular, defines boundary reactions
        met.id = str(met.id) + '1'
        met.compartment = str(met.compartment) + '1'
    
for rxn in ios2164_bs.reactions:
    rxn.id = str(rxn.id)  + '_BS'
    if "EX_" in rxn.id: #Rename media reactions
        rxn.name = "BS " + str(rxn.name)


ios2164_bs.repair()


In [21]:
read = pd.read_csv('../misc/photo_medium.csv')

#Only get M cells
photo_medium = dict(read.values)

bs_medium = dict()
for items in photo_medium:
    if "_BS" in items:
        bs_medium[items] = photo_medium[items]
        
bs_copy = copy.deepcopy(bs_medium)
bs_copy['EX_sucr_LPAREN_e_RPAREN__BS'] = 0

#Try natin may sugar export
ios2164_bs.reactions.EX_sucr_LPAREN_e_RPAREN__BS.upper_bound = 1000
ios2164_bs.reactions.EX_glc_DASH_A_LPAREN_e_RPAREN__BS.upper_bound = 1000
ios2164_bs.reactions.EX_fru_DASH_B_LPAREN_e_RPAREN__BS.upper_bound = 1000

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
bs_medium = copy.copy(bs_copy)
ios2164_bs.medium = bs_medium
bcell_rip  = riptide.contextualize(ios2164_bs, transcriptome = bscell_exp, samples= 1000)

#Apparently unconstraining the export of sucrose doesn't actually do anything.


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 472 from 2444 (80.69% change)
Metabolites pruned to 484 from 1999 (75.79% change)
Flux through the objective DECREASED to ~2.45 from ~3.02 (18.87% change)
Context-specific metabolism correlates with transcriptome (r=0.105, p=0.022 *)

RIPTiDe completed in 36 seconds



In [23]:
for rxns in ios2164_bs.exchanges:
    print(rxns.name, rxns.bounds)

BS CO2 exchange (0, 0.0)
BS H2O exchange (-1000, 1000.0)
BS H+ exchange (-1000, 1000.0)
BS nitrate exchange (-1000, 0.0)
BS O2 exchange (0, 1000.0)
BS Phosphate exchange (-1000, 1000.0)
BS Sucrose exchange (0, 1000)
BS Fructose exchange (0, 1000)
BS Glucose exchange (0, 1000)
BS Sulfate exchange (-1000, 1000.0)
BS Bicarbonate exchange (0, 0.0)
BS Sulfite exchange (0, 0.0)
BS Hydrogen sulfide exchange (0, 0.0)
BS Ammonia exchange (0, 0.0)
BS L-Asparagine exchange (0, 1000.0)
BS L-Glutamine exchange (0, 1000.0)
BS Ethanol exchange (0, 1000.0)
BS Acetate exchange (0, 1000.0)
BS L-Lactate exchange (0, 1000.0)
BS Thiosulfate exchange (0, 0.0)
BS Fe2+ exchange (-1000, 1000.0)
BS Fe3+ exchange (0, 0.0)
BS Magnesium exchange (-1000, 1000.0)
BS L-Alanine exchange (0, 1000.0)
BS L-Arginine exchange (0, 1000.0)
BS L-Aspartate exchange (0, 1000.0)
BS L-Cysteine exchange (0, 1000.0)
BS L-Glutamate exchange (0, 1000.0)
BS Glycine exchange (0, 1000.0)
BS L-Histidine exchange (0, 1000.0)
BS L-Isoleuci

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
#What if I unconstrained CO2 flux thru the BS cell? Would the context specific cells be okay?
bs_medium = copy.copy(bs_copy)

bs_medium['EX_co2_LPAREN_e_RPAREN__BS'] = 1000
ios2164_bs.medium = bs_medium

bcell_rip  = riptide.contextualize(ios2164_bs, transcriptome = bscell_exp, samples= 1000)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 445 from 2444 (81.79% change)
Metabolites pruned to 476 from 1999 (76.19% change)
Flux through the objective DECREASED to ~3.41 from ~4.24 (19.58% change)
Context-specific metabolism correlates with transcriptome (r=0.217, p<0.001 *)

RIPTiDe completed in 35 seconds



In [15]:
ios2164_bs.medium

{'EX_co2_LPAREN_e_RPAREN__BS': 1000,
 'EX_h2o_LPAREN_e_RPAREN__BS': 1000,
 'EX_h_LPAREN_e_RPAREN__BS': 1000,
 'EX_no3_LPAREN_e_RPAREN__BS': 1000,
 'EX_pi_LPAREN_e_RPAREN__BS': 1000,
 'EX_so4_LPAREN_e_RPAREN__BS': 1000,
 'EX_fe2_LPAREN_e_RPAREN__BS': 1000,
 'EX_mg2_LPAREN_e_RPAREN__BS': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__BS': 1000}

In [17]:
for item in ios2164_bs.exchanges:
    print(item.name, item.bounds)

BS CO2 exchange (-1000, 0.0)
BS H2O exchange (-1000, 1000.0)
BS H+ exchange (-1000, 1000.0)
BS nitrate exchange (-1000, 0.0)
BS O2 exchange (0, 1000.0)
BS Phosphate exchange (-1000, 1000.0)
BS Sucrose exchange (0, 0.0)
BS Fructose exchange (0, 0.0)
BS Glucose exchange (0, 0.0)
BS Sulfate exchange (-1000, 1000.0)
BS Bicarbonate exchange (0, 0.0)
BS Sulfite exchange (0, 0.0)
BS Hydrogen sulfide exchange (0, 0.0)
BS Ammonia exchange (0, 0.0)
BS L-Asparagine exchange (0, 1000.0)
BS L-Glutamine exchange (0, 1000.0)
BS Ethanol exchange (0, 1000.0)
BS Acetate exchange (0, 1000.0)
BS L-Lactate exchange (0, 1000.0)
BS Thiosulfate exchange (0, 0.0)
BS Fe2+ exchange (-1000, 1000.0)
BS Fe3+ exchange (0, 0.0)
BS Magnesium exchange (-1000, 1000.0)
BS L-Alanine exchange (0, 1000.0)
BS L-Arginine exchange (0, 1000.0)
BS L-Aspartate exchange (0, 1000.0)
BS L-Cysteine exchange (0, 1000.0)
BS L-Glutamate exchange (0, 1000.0)
BS Glycine exchange (0, 1000.0)
BS L-Histidine exchange (0, 1000.0)
BS L-Isoleuc

In [30]:
#With Sucrose import, apparently it lowers the correlation value as the cell can obtain
#Nutrients 
bs_medium = copy.copy(bs_copy)

bs_medium['EX_sucr_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_co2_LPAREN_e_RPAREN__BS'] = 1000
ios2164_bs.medium = bs_medium

bcell_rip  = riptide.contextualize(ios2164_bs, transcriptome = bscell_exp, samples= 1000)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 434 from 2444 (82.24% change)
Metabolites pruned to 461 from 1999 (76.94% change)
Flux through the objective DECREASED to ~30.88 from ~36.06 (14.36% change)
Context-specific metabolism correlates with transcriptome (r=0.104, p=0.031 *)

RIPTiDe completed in 41 seconds



In [31]:
ios2164_bs.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'EX_co2_LPAREN_e_RPAREN__BS': 1000,
 'EX_h2o_LPAREN_e_RPAREN__BS': 1000,
 'EX_h_LPAREN_e_RPAREN__BS': 1000,
 'EX_no3_LPAREN_e_RPAREN__BS': 1000,
 'EX_pi_LPAREN_e_RPAREN__BS': 1000,
 'EX_sucr_LPAREN_e_RPAREN__BS': 1000,
 'EX_so4_LPAREN_e_RPAREN__BS': 1000,
 'EX_fe2_LPAREN_e_RPAREN__BS': 1000,
 'EX_mg2_LPAREN_e_RPAREN__BS': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__BS': 1000}

In [25]:
#Gas exchange is allowed, but photoautotrophic conditions -- no sucrose import or anything

bs_medium = copy.copy(bs_copy)

bs_medium['EX_o2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_co2_LPAREN_e_RPAREN__BS'] = 1000
ios2164_bs.medium = bs_medium

bcell_rip  = riptide.contextualize(ios2164_bs, transcriptome = bscell_exp, samples= 1000)

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 461 from 2444 (81.14% change)
Metabolites pruned to 477 from 1999 (76.14% change)
Flux through the objective DECREASED to ~7.47 from ~9.25 (19.24% change)
Context-specific metabolism correlates with transcriptome (r=0.218, p<0.001 *)

RIPTiDe completed in 40 seconds



In [33]:
ios2164_bs.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'EX_co2_LPAREN_e_RPAREN__BS': 1000,
 'EX_h2o_LPAREN_e_RPAREN__BS': 1000,
 'EX_h_LPAREN_e_RPAREN__BS': 1000,
 'EX_no3_LPAREN_e_RPAREN__BS': 1000,
 'EX_o2_LPAREN_e_RPAREN__BS': 1000,
 'EX_pi_LPAREN_e_RPAREN__BS': 1000,
 'EX_so4_LPAREN_e_RPAREN__BS': 1000,
 'EX_fe2_LPAREN_e_RPAREN__BS': 1000,
 'EX_mg2_LPAREN_e_RPAREN__BS': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__BS': 1000}

In [26]:
#Try adding some ammonia into the medium

bs_medium = copy.copy(bs_copy)

bs_medium['EX_o2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_co2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_nh4_LPAREN_e_RPAREN__BS'] = 1000
ios2164_bs.medium = bs_medium

bcell_rip  = riptide.contextualize(ios2164_bs, transcriptome = bscell_exp, samples= 1000)

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 458 from 2444 (81.26% change)
Metabolites pruned to 475 from 1999 (76.24% change)
Flux through the objective DECREASED to ~8.67 from ~9.28 (6.57% change)
Context-specific metabolism correlates with transcriptome (r=0.226, p<0.001 *)

RIPTiDe completed in 41 seconds



In [35]:
ios2164_bs.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'EX_co2_LPAREN_e_RPAREN__BS': 1000,
 'EX_h2o_LPAREN_e_RPAREN__BS': 1000,
 'EX_h_LPAREN_e_RPAREN__BS': 1000,
 'EX_no3_LPAREN_e_RPAREN__BS': 1000,
 'EX_o2_LPAREN_e_RPAREN__BS': 1000,
 'EX_pi_LPAREN_e_RPAREN__BS': 1000,
 'EX_so4_LPAREN_e_RPAREN__BS': 1000,
 'EX_nh4_LPAREN_e_RPAREN__BS': 1000,
 'EX_fe2_LPAREN_e_RPAREN__BS': 1000,
 'EX_mg2_LPAREN_e_RPAREN__BS': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__BS': 1000}

In [27]:
#With a single AA, test lang haha
bs_medium = copy.copy(bs_copy)

bs_medium['EX_o2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_co2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_nh4_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_ala_DASH_L_LPAREN_e_RPAREN__BS'] = 1000
ios2164_bs.medium = bs_medium

bcell_rip  = riptide.contextualize(ios2164_bs, transcriptome = bscell_exp, samples= 1000)


Initializing model and integrating transcriptomic data...


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 450 from 2444 (81.59% change)
Metabolites pruned to 470 from 1999 (76.49% change)
Flux through the objective DECREASED to ~9.07 from ~11.0 (17.55% change)
Context-specific metabolism correlates with transcriptome (r=0.123, p=0.009 *)

RIPTiDe completed in 39 seconds



In [28]:
#How about with AAs instead?
bs_medium = copy.copy(bs_copy)

bs_medium['EX_o2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_co2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_nh4_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_ala_DASH_L_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_arg_DASH_L_LPAREN_e_RPAREN__BS'] = 1000
ios2164_bs.medium = bs_medium

bcell_rip  = riptide.contextualize(ios2164_bs, transcriptome = bscell_exp, samples= 1000)


Initializing model and integrating transcriptomic data...


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 445 from 2444 (81.79% change)
Metabolites pruned to 464 from 1999 (76.79% change)
Flux through the objective DECREASED to ~8.19 from ~11.0 (25.55% change)
Context-specific metabolism correlates with transcriptome (r=0.154, p=0.001 *)

RIPTiDe completed in 40 seconds



In [29]:
#How about with FE3 import?
bs_medium = copy.copy(bs_copy)

bs_medium['EX_o2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_co2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_fe3_LPAREN_e_RPAREN__BS'] = 1000

ios2164_bs.medium = bs_medium

bcell_rip  = riptide.contextualize(ios2164_bs, transcriptome = bscell_exp, samples= 1000)


Initializing model and integrating transcriptomic data...


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 461 from 2444 (81.14% change)
Metabolites pruned to 478 from 1999 (76.09% change)
Flux through the objective DECREASED to ~7.72 from ~9.25 (16.54% change)
Context-specific metabolism correlates with transcriptome (r=0.218, p<0.001 *)

RIPTiDe completed in 41 seconds



In [30]:
#How about with FE3 import?
bs_medium = copy.copy(bs_copy)

bs_medium['EX_o2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_co2_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_fe3_LPAREN_e_RPAREN__BS'] = 1000
bs_medium['EX_nh4_LPAREN_e_RPAREN__BS'] = 1000


ios2164_bs.medium = bs_medium

bcell_rip  = riptide.contextualize(ios2164_bs, transcriptome = bscell_exp, samples= 1000)


Initializing model and integrating transcriptomic data...


/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 457 from 2444 (81.3% change)
Metabolites pruned to 475 from 1999 (76.24% change)
Flux through the objective DECREASED to ~9.19 from ~9.28 (0.97% change)
Context-specific metabolism correlates with transcriptome (r=0.227, p<0.001 *)

RIPTiDe completed in 42 seconds



In [32]:
ios2164_bs.medium

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'EX_co2_LPAREN_e_RPAREN__BS': 1000,
 'EX_h2o_LPAREN_e_RPAREN__BS': 1000,
 'EX_h_LPAREN_e_RPAREN__BS': 1000,
 'EX_no3_LPAREN_e_RPAREN__BS': 1000,
 'EX_o2_LPAREN_e_RPAREN__BS': 1000,
 'EX_pi_LPAREN_e_RPAREN__BS': 1000,
 'EX_so4_LPAREN_e_RPAREN__BS': 1000,
 'EX_nh4_LPAREN_e_RPAREN__BS': 1000,
 'EX_fe2_LPAREN_e_RPAREN__BS': 1000,
 'EX_mg2_LPAREN_e_RPAREN__BS': 1000,
 'EX_photonVis_LPAREN_e_RPAREN__BS': 1000}

In [34]:
for item in bcell_rip.model.exchanges:
    print(item.name, item.bounds)

BS CO2 exchange (-117.62971448203314, 0.0)
BS H2O exchange (465.81919906090104, 1000.0)
BS H+ exchange (-878.5957681926998, -351.80183108839367)
BS O2 exchange (-540.5758412818209, -260.74599101929937)
BS Phosphate exchange (-1.8973466062176385, -1.5178772849858433)
BS Sulfate exchange (-0.10021698660978018, -0.08017358928805152)
BS Ammonia exchange (-5.8933104658018465, -4.714648372663532)
BS photon emission (-1000.0, -269.24075180839014)


Benchmarking results for BS Cells:

Unconstrained CO2 Flux thru BS: r=0.114, p<0.013 (Less significant)
constrained CO2 Flux thru the BS cells: r=0.192 , p<0.001 (Highly significant)
If I add Sucrose to the medium: r=0.096; p=0.045 (Low significance)
No sucrose, only O2 and CO2: r=0.205, p<0.001 (High significance)
With NH4, Gas exchange: r=0.245, p<0.001
With Gas exchange + a single AA: r=0.134, p=0.005
With Gas exchange + alanine + arg: r=0.148, p=0.
With NH4, Gas exchange +  Fe3 import: r=0.0555, n.s.
low light conditions (500 ppfd) + Gas + NH4 = r=0.094 p=0.045 


Notes:
Setting up monosaccharide export didn't do much in terms of correlation results.
Okay, select the medium with NH4 + Gas exchange instead for max correlation////